In [50]:
import cv2

if hasattr(cv2, "aruco"):
    print("Aruco is available.")


Aruco is available.


In [51]:
# %% ─────────────  QR-DRIVEN ArUco overlay (minimal)  ─────────────
import cv2
import json
import numpy as np
import logging

VIDEO_IN = "aruco.mp4"
VIDEO_OUT = "aruco_green_overlay.mp4"
ALPHA = 0.30  # overlay transparency

# ── logging (still handy) ──────────────────────────────────────────
logging.basicConfig(
    filename="aruco_debug.log", filemode="w", level=logging.DEBUG, format="%(message)s"
)
log = logging.getLogger("aruco")

# ═════ PASS-0 : read the metadata QR ═══════════════════════════════
cap = cv2.VideoCapture(VIDEO_IN)
qr = cv2.QRCodeDetector()

meta = None
while True:
    ok, frm = cap.read()
    if not ok:
        break  # end of video
    payload, *_ = qr.detectAndDecode(frm)
    if payload:
        meta = json.loads(payload)
        break

if meta is None:
    raise RuntimeError("No metadata QR code found in the video")

for k in (
    "dictionary",
    "margin_px",
    "marker_size_px",
    "corner_markers",
    "screen_aspect",
):
    if k not in meta:
        raise RuntimeError(f"QR metadata missing key: {k}")

# ── constants from the QR ─────────────────────────────────────────
dict_name = meta["dictionary"]
dictionary = cv2.aruco.getPredefinedDictionary(getattr(cv2.aruco, dict_name))
margin_px = int(meta["margin_px"])

corner_map = {  # marker-id ➜ marker-corner-index (0-3)
    meta["corner_markers"]["TL"]["id"]: meta["corner_markers"]["TL"]["corner_index"],
    meta["corner_markers"]["TR"]["id"]: meta["corner_markers"]["TR"]["corner_index"],
    meta["corner_markers"]["BR"]["id"]: meta["corner_markers"]["BR"]["corner_index"],
    meta["corner_markers"]["BL"]["id"]: meta["corner_markers"]["BL"]["corner_index"],
}

# neighbours inside one marker (OpenCV order 0-1-2-3)
NEIGH_H = {0: 1, 1: 0, 2: 3, 3: 2}
NEIGH_V = {0: 3, 1: 2, 2: 1, 3: 0}


def screen_corner_from_marker(c4, inner):
    """
    Move the detected *inner* corner diagonally outward by `margin_px`
    along the marker edges to reach the real screen corner.
    """
    c = c4[inner]
    h = c4[NEIGH_H[inner]]
    v = c4[NEIGH_V[inner]]
    u_h = (c - h) / np.linalg.norm(c - h)
    u_v = (c - v) / np.linalg.norm(c - v)
    diag = u_h + u_v
    diag /= np.linalg.norm(diag)  # unit vector toward screen corner
    return c + margin_px * diag


# ═════ PASS-1 : process & draw frame-by-frame ═════════════════════
fps = cap.get(cv2.CAP_PROP_FPS)
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
cap.set(cv2.CAP_PROP_POS_FRAMES, 0)  # rewind

fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(VIDEO_OUT, fourcc, fps, (w, h))
detector = cv2.aruco.ArucoDetector(dictionary, cv2.aruco.DetectorParameters())

while True:
    ok, frame = cap.read()
    if not ok:
        break
    quad = np.zeros((4, 2), np.float32)
    detected = [False] * 4

    corners, ids, _ = detector.detectMarkers(frame)
    if ids is not None:
        for idx, mid in enumerate(ids.flatten()):
            if mid in corner_map:
                i = corner_map[mid]
                quad[i] = screen_corner_from_marker(corners[idx][0], i)
                detected[i] = True

    # draw only if **all 4** corners were found in this frame
    if all(detected):
        poly = quad.astype(np.int32)
        mask = np.zeros_like(frame)
        cv2.fillPoly(mask, [poly], (0, 255, 0))
        frame = cv2.addWeighted(frame, 1 - ALPHA, mask, ALPHA, 0)
        cv2.polylines(frame, [poly], True, (0, 255, 0), 2)

    out.write(frame)

cap.release()
out.release()
print("✅  Finished – saved to", VIDEO_OUT)


✅  Finished – saved to aruco_green_overlay.mp4
